## Pytorch의 두가지 주요한 특징

* Numpy와 유사하지만 GPU 상에서 실행 가능한 n-차원 텐서(Tensor)
* 신경망을 구성하고 학습하는 과정에서의 자동 미분(Automatic differentiation)

## Numpy를 사용한 신경망

In [4]:
import numpy as np
import math


# 무작위로 입력과 출력 데이터를 생성
x = np.linspace(-math.pi, math.pi, 2000)
y = np.sin(x)


# 무작위로 가중치 초기화
a = np.random.randn()
b = np.random.randn()
c = np.random.randn()
d = np.random.randn()

learning_rate = 1e-6
for t in range(2000):
    # 순전파 단계 : 예측값 y 계산
    # y = a + bx + cx^2 + dx^3
    y_pred = a + b * x + c * x ** 2 + d * x ** 3
    
    # 손실(loss)을 계산하고 출력
    loss = np.square(y_pred - y).sum()
    if t % 100 == 99:
        print(t, loss)
        
    # 손실에 따른 a, b, c, d의 변화도(gradient)를 계산하고 역전파
    grad_y_pred = 2.0 * (y_pred - y)
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    grad_d = (grad_y_pred * x ** 3).sum()
    
    # 가중치를 갱신
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d
    
print(f'Result: y = {a} + {b}x + {c}x^2 + {d}x^3')

99 2874.528589900541
199 1904.9289225133243
299 1263.4433682415024
399 839.0206003943701
499 558.2009095060077
599 372.3885024874886
699 249.43486352094132
799 168.0714930656634
899 114.2273124130329
999 78.59266718937205
1099 55.00790184747686
1199 39.39736386692083
1299 29.06420222207438
1399 22.22383224305969
1499 17.695283146119806
1599 14.696991128824669
1699 12.711689395183734
1799 11.397011380336828
1899 10.52633799991306
1999 9.9496552033279
Result: y = 0.006266679661426255 + 0.8245379557300995x + -0.0010811065195957055x^2 + -0.08874980874116986x^3


## Tensor를 사용한 신경망

In [6]:
import torch
import math

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # GPU에서 실행하려면 해당 주석 제거

# 무작위로 입력과 출력 데이터를 생성
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

# 무작위로 가중치 초기화
a = torch.randn((), device=device, dtype=dtype)
b = torch.randn((), device=device, dtype=dtype)
c = torch.randn((), device=device, dtype=dtype)
d = torch.randn((), device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(2000):
    # 순전파 단계
    y_pred = a + b*x + c*x**2 + d*x**3
    
    #손실(loss) 계산 및 출력
    loss = (y_pred - y).pow(2).sum().item()
    if t % 100 == 99:
        print(t, loss)
        
    # 손실에 따른 a, b, c, d의 변화도(gradient) 계산하고 역전파
    grad_y_pred = 2.0 * (y_pred - y)
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred*x).sum()
    grad_c = (grad_y_pred*x**2).sum()
    grad_d = (grad_y_pred*x**3).sum()
    
    # 가중치 갱신
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d
    
    
print(f'Result: y = {a.item()} + {b.item()}x + {c.item()}x^2 + {d.item()}x^3')

99 787.5328979492188
199 537.771240234375
299 368.5162658691406
399 253.6938934326172
499 175.7135467529297
599 122.69551849365234
699 86.60891723632812
799 62.0188102722168
899 45.243560791015625
999 33.786476135253906
1099 25.952495574951172
1199 20.589712142944336
1299 16.914430618286133
1399 14.392740249633789
1499 12.660589218139648
1599 11.46943187713623
1699 10.649398803710938
1799 10.08421516418457
1899 9.694269180297852
1999 9.424943923950195
Result: y = 0.021788645535707474 + 0.8435530662536621x + -0.0037589017301797867x^2 + -0.09145454317331314x^3


## Autograd
* Autograd를 사용하면, 신경망의 순전파 단계에서 연산 그래프(computational graph)를 정의한다
* 이 그래프의 node는 tensor이고, edge는 입력 텐서로부터 출력 텐서를 만들어내는 함수

In [8]:
import torch
import math

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0")

# 입력값과 출력값을 갖는 텐서들 생성
# requires_grad=False가 기본값으로 설정되어 역전파 단계 중에 이 텐서들에 대한 변화도를 계산할 필요가 없음을 나타냄
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)


# 가중치를 갖는 임의의 텐서 생성
# y = a + bx + cx^2 + dx^3
# requires_grad = True로 설정하여 역전파 단계 중에 이 텐서들에 대한 변화도를 계산할 필요가 있음을 나타냄
a = torch.randn((), device=device, dtype=dtype, requires_grad=True)
b = torch.randn((), device=device, dtype=dtype, requires_grad=True)
c = torch.randn((), device=device, dtype=dtype, requires_grad=True)
d = torch.randn((), device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(2000):
    # 순전파 단계
    y_pred = a + b*x + c*x**2 + d*x**3
    
    
    # 텐서들간의 연산을 사용하여 손실(loss)를 계산하고 출력
    # 이 때 손실은 (1, ) shape를 갖는 텐서
    # loss.item() 으로 loss가 갖고 있는 스칼라 값을 가져올 수 있음
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())
        
    # autograd를 사용하여 역전파 단계를 계산
    # requires_grad=True를 갖는 모든 텐서들에 대한 손실의 변화도를 계산
    loss.backward()
    
    # Gradient Descent를 사용하여 가중치를 직접 갱신
    # torch.no_grad()로 감싸는 이유는, 가중치들이 requires_grad=True지만 autograd에서는 이를 추적하지 않을 것이기 때문
    with torch.no_grad():
        a -= learning_rate * a.grad
        b -= learning_rate * b.grad
        c -= learning_rate * c.grad
        d -= learning_rate * d.grad
        
        # 가중치 갱신 후에는 변화도를 직접 0으로 만듦
        
        a.grad = None
        b.grad = None
        c.grad = None
        d.grad = None
        
print(f'Result: y = {a.item()} + {b.item()}x + {c.item()}x^2 + {d.item()}')

99 1592.4354248046875
199 1107.1456298828125
299 771.290771484375
399 538.623046875
499 377.28143310546875
599 265.29412841796875
699 187.4916534423828
799 133.3904571533203
899 95.73783874511719
999 69.51084899902344
1099 51.227691650390625
1199 38.47236251831055
1299 29.56684112548828
1399 23.344715118408203
1499 18.99447250366211
1599 15.95096492767334
1699 13.820333480834961
1799 12.327840805053711
1899 11.281784057617188
1999 10.548212051391602
Result: y = 0.04160797968506813 + 0.8434882164001465x + -0.007178067695349455x^2 + -0.09144531935453415


## Pytorch: 새 autograd Function 정의
내부적으로, autograd의 기본(primitive) 연산자는 실제로 텐서를 조작하는 2개의 함수.
forward 함수는 입력 텐서로부터 출력텐서를 계산
backward 함수는 어떤 스칼라 값에 대한 출력 텐서의 변화도(gradient)를 전달

이 예제에서는 y = a + bx + cx^2 + dx^3 대신 y = a + bP3(c + dx)로 모델을 정의
여기서 P3(x)= 1/2(5x^3-3x)은 3차 르장드르 다항식

In [10]:
import torch
import math

class LegendrePolynomial3(torch.autograd.Function):
    # torch.autograd.Function을 상속받아 사용자 정의 autograd Function을 구현하고, 텐서 연산을 하는 순전파 단계와 역전파 단게를 구현
    
    @staticmethod
    def forward(ctx, input):
        """
        순전파 단계에서는 입력을 갖는 텐서를 받아 출력을 갖는 텐서를 반환
        ctx는 컨텍스트 객체(context object)로 역전파 연산을 위한 정보 저장에 사용
        ctx.save_for_backward 메소드를 사용하여 역전파 단게에서 사용할 어떤 객체도 저장(cache) 가능
        """
        ctx.save_for_backward(input)
        return 0.5 * (5 * input ** 3 - 3 * input)
    
    @staticmethod
    def backward(ctx, grad_output):
        """
        역전파 단계에서는 출력에 대한 손실(loss)의 변화도(gradient)를 갖는 텐서를 받고, 
        입력에 대한 손실의 변화도를 계산해야함
        """
        input, = ctx.saved_tensors
        return grad_output * 1.5 * (5 * input ** 2 - 1)
    
dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0")

# 입력값과 출력값을 갖는 텐서들을 생성
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

# 가중치를 갖는 임의의 텐서를 생성
# y = a + b*P3(c + d*x)
# 이 가중치들이 수렴(convergence)하기 위해서는 정답으로부터 너무 멀리 떨어지지 않은 값으로 초기화가 되어야 함
a = torch.full((), 0.0, device=device, dtype=dtype, requires_grad=True)
b = torch.full((), -1.0, device=device, dtype=dtype, requires_grad=True)
c = torch.full((), 0.0, device=device, dtype=dtype, requires_grad=True)
d = torch.full((), 0.3, device=device, dtype=dtype, requires_grad=True)

learning_rate = 5e-6
for t in range(2000):
    # 사용자 정의 Function을 적용하기 위해 Function.apply 메소드 사용
    # 여기에 'P3'라고 이름을 붙임
    P3 = LegendrePolynomial3.apply
    
    # 순전파 단계
    # 사용자 정의 autograd 연산을 사용하여 P3를 계산
    y_pred = a + b * P3(c + d * x)
    
    # 손실을 계산하고 출력
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())
        
    # autograd를 사용하여 역전파 단계를 계산
    loss.backward()
    
    # 경사하강법(gradient descent)을 사용하여 가중치를 갱신
    with torch.no_grad():
        a -= learning_rate * a.grad
        b -= learning_rate * b.grad
        c -= learning_rate * c.grad
        d -= learning_rate * d.grad

        # 가중치 갱신 후에 변화도를 0으로 초기화
        a.grad = None
        b.grad = None
        c.grad = None
        d.grad = None
        
print(f'Result: y ={a.item()} + {b.item()}x + {c.item()}x^2 | {d.item()}x^3')

99 209.95834350585938
199 144.66018676757812
299 100.70249938964844
399 71.03519439697266
499 50.97850799560547
599 37.403133392333984
699 28.206867218017578
799 21.973188400268555
899 17.7457275390625
999 14.877889633178711
1099 12.931766510009766
1199 11.610918045043945
1299 10.714258193969727
1399 10.10548210144043
1499 9.692106246948242
1599 9.411375999450684
1699 9.220745086669922
1799 9.091285705566406
1899 9.003361701965332
1999 8.943639755249023
Result: y =-6.8844756562214116e-09 + -2.208526849746704x + 1.5037101563919464e-09x^2 | 0.2554861009120941x^3


## nn 모듈
nn패키지는 신경망 계층(layer)와 거의 비슷한 Module의 집합을 정의
Module은 입력 텐서를 받고 출력 텐서를 계산하는 한편, 학습 가능한 매개변수를 갖는 텐서들을 내부 상태(internal state)로 가짐
nn 패키지는 또한 신경망을 학습시킬 때 주로 사용하는 유용한 손실 함수(loss function)들도 정의하고 있음

In [19]:
import torch
import math

x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# 이 예제에서, 출력 y는 (x, x^2, x^3)의 선형 함수이므로, 선형 계층 신경망으로 간주 할 수 있음
# (x, x^2, x^3)를 위한 텐서를 준비
p = torch.tensor([1, 2, 3])
xx = x.unsqueeze(-1).pow(p)


# nn 패키지를 사용하여 모델을 순차적 계층(sequence of layers)으로 정의
# nn.Sequential은 다른 module을 포함하는 Module로, 포함되는 Module들을 순차적으로 적용하여 출력을 생성
# 각각의 Linear Module은 선형 함수(linear function)를 사용하여 입력으로부터 출력을 계산하고, 내부 Tensor에 가중치와 편향을 저장
# Flatten 계층은 선형 게층의 출력을 'y'의 shape과 맞도록 1D 텐서로 편다(flatten)

model = torch.nn.Sequential(
    torch.nn.Linear(3, 1),
    torch.nn.Flatten(0, 1)
)

loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-6
for t in range(2000):
    y_pred = model(xx)
    
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())
        
    model.zero_grad()
    
    loss.backward()
    
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad
            
linear_layer = model[0]

print(f'Result: y = {linear_layer.bias.item()} + {linear_layer.weight[:, 0].item()} x + {linear_layer.weight[:, 1].item()} x^2 + {linear_layer.weight[:, 2].item()} x^3')

99 323.89605712890625
199 226.08961486816406
299 158.8056182861328
399 112.46723175048828
499 80.51905822753906
599 58.468597412109375
699 43.233299255371094
799 32.69586181640625
899 25.400239944458008
999 20.3441104888916
1099 16.83662223815918
1199 14.401142120361328
1299 12.708513259887695
1399 11.531079292297363
1499 10.711360931396484
1599 10.140201568603516
1699 9.741915702819824
1799 9.463964462280273
1899 9.269845008850098
1999 9.134183883666992
Result: y = 0.01738000474870205 + 0.8500474095344543 x + -0.002998340642079711 x^2 + -0.09237831085920334 x^3


## Pytorch: optim
Pytorch의 optim 패키지는 최적화 알고리즘에 대한 아이디어를 추상화하고 일반적으로 사용하는 최적화 알고리즘의 구현체(implementation)를 제공

In [20]:
import torch
import math

x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

p = torch.tensor([1, 2, 3])
xx = x.unsqueeze(-1).pow(p)

model = torch.nn.Sequential(
    torch.nn.Linear(3, 1),
    torch.nn.Flatten(0, 1)
)
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-3
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
for t in range(2000):
    y_pred = model(xx)
    
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())
        
    optimizer.zero_grad()
    
    loss.backward()
    
    optimizer.step()
    
linear_layer = model[0]
print(f'Result: y = {linear_layer.bias.item()} + {linear_layer.weight[:, 0].item()} x + {linear_layer.weight[:, 1].item()} x^2 + {linear_layer.weight[:, 2].item()} x^3')

99 826.1966552734375
199 306.29034423828125
299 266.5346984863281
399 227.42237854003906
499 179.69998168945312
599 129.66229248046875
699 84.90664672851562
799 50.38207244873047
899 27.479511260986328
999 14.997322082519531
1099 10.037863731384277
1199 8.918424606323242
1299 9.051300048828125
1399 8.848127365112305
1499 8.925853729248047
1599 9.055133819580078
1699 8.91148567199707
1799 8.910625457763672
1899 8.92923641204834
1999 8.926820755004883
Result: y = -0.0004426115774549544 + 0.8562188744544983 x + -0.00044262089068070054 x^2 + -0.09385290741920471 x^3


## Pytorch: 사용자 정의 nn.Module
기존 Module의 구성(sequence)보다 더 복잡한 모델을 구성해야할 때, nn.Module의 하위 클래스로 새로운 Module을 정의하고, 입력 텐서를 받아 다른 모듈 및 autograd 연산을 사용하여 출력 텐서를 만드는 forward를 정의

In [23]:
import torch
import math

class Polynomial3(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.a = torch.nn.Parameter(torch.randn(()))
        self.b = torch.nn.Parameter(torch.randn(()))
        self.c = torch.nn.Parameter(torch.randn(()))
        self.d = torch.nn.Parameter(torch.randn(()))
        
    def forward(self, x):
        """
        텐서들 간의 임의의 연산뿐만 아니라, 생성자에서 정의한 Module을 사용할 수 있음
        """
        return self.a + self.b*x + self.c * x ** 2 + self.d * x ** 3
    
    def string(self):
        """
        Python의 다른 클래스처럼, Pytorch 모듈을 사용해서 사용자 정의 메소드를 정의할 수 있음
        """
        return f'y = {self.a.item()} + {self.b.item()} x + {self.c.item()} x^2 + {self.d.item()} x^3'
    

x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

model = Polynomial3()

criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-6)
for t in range(2000):
    y_pred = model(x)
    
    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())
        
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
print(f'Result: {model.string()}')

99 349.53533935546875
199 235.78836059570312
299 160.0872039794922
399 109.68301391601562
499 76.10843658447266
599 53.733619689941406
699 38.815433502197266
799 28.863845825195312
899 22.221914291381836
999 17.78640365600586
1099 14.822625160217285
1199 12.84101390838623
1299 11.51526165008545
1399 10.627704620361328
1499 10.033076286315918
1599 9.634425163269043
1699 9.366943359375
1799 9.187325477600098
1899 9.066634178161621
1999 8.985441207885742
Result: y = -0.007527928799390793 + 0.8672866821289062 x + 0.0012986946385353804 x^2 + -0.09483043849468231 x^3


## Pytorch: 제어 흐름(Control Flow) + 가중치 공유(Weight Sharing)

In [25]:
import random
import torch
import math

class DynamicNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.a = torch.nn.Parameter(torch.randn(()))
        self.b = torch.nn.Parameter(torch.randn(()))
        self.c = torch.nn.Parameter(torch.randn(()))
        self.d = torch.nn.Parameter(torch.randn(()))
        self.e = torch.nn.Parameter(torch.randn(()))
        
    def forward(self, x):
        y = self.a + self.b*x + self.c * x**2 + self.d * x **3
        for exp in range(4, random.randint(4, 6)):
            y = y + self.e * x ** exp
        return y
    
    def string(self):
        return f'y = {self.a.item()} + {self.b.item()} x + {self.c.item()} x^2 + {self.d.item()} x^3 + {self.e.item()} x^4 ? + {self.e.item()} x^5 ?'

x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

model = DynamicNet()

criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-8, momentum=0.9)
for t in range(30000):
    y_pred = model(x)
    
    loss = criterion(y_pred, y)
    
    if t % 2000 == 1999:
        print(t, loss.item())
        
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
print(f'Result: {model.string()}')

1999 3248.16796875
3999 1496.657958984375
5999 741.333251953125
7999 382.6327209472656
9999 217.3163299560547
11999 88.50534057617188
13999 48.05274963378906
15999 29.503379821777344
17999 60.692138671875
19999 13.505494117736816
21999 11.250572204589844
23999 9.78499984741211
25999 9.422563552856445
27999 9.121001243591309
29999 8.631287574768066
Result: y = 0.013029329478740692 + 0.8549340963363647 x + -0.0028773683588951826 x^2 + -0.09358595311641693 x^3 + 0.00015116340364329517 x^4 ? + 0.00015116340364329517 x^5 ?
